In [1]:
#@title Configuration
from google.colab import drive
import os
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf /content/drive/Shareddrives/DA231-2021-Aug-Public/spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"


Mounted at /content/drive


In [2]:
#@title Initialization { display-mode: "both" }
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
         .master("local")\
         .appName("Colab")\
         .config('spark.ui.port', '4050')\
         .getOrCreate()
spark

In [3]:
#@title Common Imports
from pyspark import SparkContext, SparkConf
import sys
import json
import pyspark.sql.functions as F
sc = SparkContext.getOrCreate()
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import widgets
from IPython.display import display

from math import sqrt
from os.path import exists

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import Evaluator
from pyspark.ml.stat import Correlation

File path, Folder ID, Constants and other related information

In [4]:
#@title File path, Folder ID, Constants
drivepath = "/content/drive/Shareddrives/ProjectSharedDrive/wazirxCSVData"
driveDAILYpath = "/content/drive/Shareddrives/ProjectSharedDrive/wazirxCSVCumulative"
cryptofile = "/content/drive/Shareddrives/ProjectSharedDrive/top25_cryptocoins_list.jsonl"


In [5]:
#@title Load Top 10 Coin Details
fileRef = open(driveDAILYpath+"/USDTCoinName.txt" ,"r")
coinName = []
coinCode = []
for lines in fileRef:
  name = eval(lines[:-1])
  coinName.append(name[1])
  coinCode.append(name)

_crypt_raw = spark.read.json(cryptofile)
_crypt = _crypt_raw.withColumn("name", F.col("name")) \
          .withColumn("symbol", F.col("symbol")) \
          .withColumn("cmc_rank", F.col("cmc_rank")).orderBy("cmc_rank")
#_crypt.show()
top10=[]
j=0
for i in _crypt.collect():
   top10.append(i[1])
   j+=1
   if(j==10):
     break

In [6]:
#@title Non top 10 Coins collections

fileRef = open(driveDAILYpath+"/USDTCoinName.txt" ,"r")
coinName = []
coinCode = []
for lines in fileRef:
  name = eval(lines[:-1])
  coinName.append(name[1])
  coinCode.append(name)

#print(len(coinName))
coinName = [x for x in coinName if x not in top10]

## **Interactive Display for Trend Trading Indicators**

Choose the coin to start processing

In [7]:
#@title Select the Coin
#TTIVal = ["Moving Average", "MACD", "RSI", "Stochastic RSI", "Standard Deviation"]

coin = widgets.Dropdown(
    options=coinName,
    value=coinName[0],
    description='Select Coin: ',
    disabled=False,
)

display(coin)

Dropdown(description='Select Coin: ', options=('TRON', 'EOS', 'ICON', 'Litecoin', 'TrueUSD', 'Hydro Protocol',…


Processing the selected coin. Run the next columns for Linear Regression Model

In [8]:
#@title Perform the Linear Regression
coinChosen = [item for item in coinCode if item[1] == coin.value]
coinname = coin.value
print(coinname)
coinfile = driveDAILYpath+"/"+coinChosen[0][0]+"DAILY.csv"


p0 = spark.read.option("header",True).option("inferSchema",True).csv(coinfile)
p2 = p0.drop("_c0").drop("Open").drop("High").drop("Volume").drop("Low")
#p2.show()
for i in top10:
  coinChosen = [item for item in coinCode if item[1] == i]
  coinname = i
  #print(coinname)
  coinfile = driveDAILYpath+"/"+coinChosen[0][0]+"DAILY.csv"
  #print(coinfile)
  #coinn = "coindf"+i
  coinn = spark.read.option("header",True).option("inferSchema",True).csv(coinfile)
  topcoindf = coinn.drop("_c0").drop("Open").drop("High").drop("Volume").drop("Low").withColumnRenamed("Close",i).withColumnRenamed("Date","top_date")
  #topcoin.show()
  
  p2 = p2.join(topcoindf,(p2.Date==topcoindf.top_date)).drop("top_date")
  #p2.show()
#p2.show()
class MAPEEvaluator(Evaluator): # TODO: Fix class inheritance
  def __init__(self, predictionCol="prediction", labelCol="label"):
    self.predictionCol = predictionCol
    self.labelCol = labelCol

  def _evaluate(self, dataset):
    return evaluate(dataset, predictionCol, labelCol)

  def evaluate(dataset, predictionCol="prediction", labelCol="label"): # FIXME
    mapeDF = dataset.withColumn("ape", (F.col(predictionCol)-F.col(labelCol)) / F.col(labelCol))  \
      .groupBy().avg("ape").select("avg(ape)").withColumnRenamed("avg(ape)", "mape")
    r = mapeDF.collect()
    answer = [(row.mape) for row in r]
    return abs(answer[0]) * 100

inputTrainDF, inputTestDF = p2.randomSplit([0.8,0.2], seed=1947) 
vecAss = VectorAssembler(inputCols=top10,outputCol="features")
rf = LinearRegression(featuresCol="features",labelCol="Close")
pipeline = Pipeline(stages=[vecAss,rf]) 
model = pipeline.fit(inputTrainDF)
predDF = model.transform(inputTestDF)

    # Create evaluator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Close")
r2 = evaluator.evaluate(predDF, {evaluator.metricName: "r2"})
# evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
#rmse = evaluator.evaluate(predDF, {evaluator.metricName: "mae"})
#print(rmse)
mae = evaluator.evaluate(predDF, {evaluator.metricName: "mae"})
rmse = evaluator.evaluate(predDF, {evaluator.metricName: "rmse"})
mape = MAPEEvaluator.evaluate(predDF, predictionCol="prediction", labelCol="Close")
print(f"{r2}, {mape}, {mae}, {rmse}")
#p2.show()

TRON
0.6100070312444255, 1.2288993990474208, 0.004671169083199578, 0.005840088539081689
